<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/SuperPiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SUPER PIANO (XLNet Piano)

<a href="https://colab.research.google.com/github/asigalov61/SuperPiano/blob/master/SuperPiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Huge thanks goes to Andrew Shaw (bearpelican) for the MusicAutoBot repo/code that was used to create this notebook. This notebook is a fork of Andrew's notebook for MusicTransformer

https://github.com/bearpelican

https://github.com/bearpelican/musicautobot

# SETUP THE ENVIRONMENT

In [ ]:
#@title Install all dependencies/clone repos/import modules
!git clone https://github.com/asigalov61/musicautobot.git
!nvidia-smi
!apt install fluidsynth
!gsutil -q -m cp gs://magentadata/soundfonts/Yamaha-C5-Salamander-JNv5.1.sf2 /content/musicautobot
!pip install torch fastai music21 pebble fluidsynth midi2audio
!pip install pretty_midi
!pip install pypianoroll
!pip install mir_eval

import os
os.chdir('musicautobot')
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.numpy_encode import stream2npenc_parts
from musicautobot.utils.midifile import *
from fastai.text import *
from midi2audio import FluidSynth
from IPython.display import Audio
# Colab cannot play music directly from music21 - must convert to .wav first
def play_wav(stream):
    out_midi = stream.write('midi')
    out_wav = str(Path(out_midi).with_suffix('.wav'))
    FluidSynth("/content/musicautobot/Yamaha-C5-Salamander-JNv5.1.sf2").midi_to_audio(out_midi, out_wav)
    return Audio(out_wav)
from google.colab import files


In [ ]:
#@title Mount your Google Drive (Highly recommended)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title (OPTIONAL) Download SuperPiano original and ready-to-use training data and model
!wget 'https://superpiano.s3-us-west-1.amazonaws.com/training-data-piano-no-dynamics.pkl'
!wget 'https://superpiano.s3-us-west-1.amazonaws.com/SuperPianoModel.pth'

# PREP TRAINING DATA

In [ ]:
#@title Initialize the paths and variables
# Location of your midi filesfiles
midi_path = Path('/content/musicautobot/data/midi')
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path('/content/musicautobot')
data_path.mkdir(parents=True, exist_ok=True)

# Location of preprocessed numpy files
numpy_path = Path('/content/musicautobot/lm')

# Location to save trained model/take saved model checkpoint
model_path = Path('/content/musicautobot')

data_save_name = 'training-data-piano-no-dynamics.pkl'
[p.mkdir(parents=True, exist_ok=True) for p in [midi_path, numpy_path, data_path, model_path]];

#Upload your own MIDI files or dowload ready-to-use DataSet

In [ ]:
#@title Upload Files Here
from google.colab import files
uploaded = files.upload()

In [ ]:
#@title (The Best Choice/Works best stand-alone) Super Piano 2 Original 2500 MIDIs of Piano Music
%cd /content/musicautobot/data/midi
!wget 'https://github.com/asigalov61/SuperPiano/raw/master/Super_Piano_2_MIDI_DataSet_CC_BY_NC_SA.zip'
!unzip -j 'Super_Piano_2_MIDI_DataSet_CC_BY_NC_SA.zip'

In [ ]:
#@title (Second Best Choice/Works best stand-alone) Alex Piano Only Drafts Original 1500 MIDIs 
%cd /content/musicautobot/data/midi
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-All-Drafts-Piano-Only.zip'

In [ ]:
#@title (Optional) Google Magenta MAESTRO Piano MIDI Dataset (~1300 MIDIs)
%cd /content/musicautobot/data/midi
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip

#MIDI PROCESSING

In [ ]:
#@title Calculate and display number of indexed MIDI files
# num_tracks = [1, 2] # number of tracks to support
cutoff = 5 # max instruments
min_variation = 3 # minimum number of different midi notes played
# max_dur = 128

midi_files = get_files(midi_path, '.midi', recurse=False); len(midi_files)

In [ ]:
#@title Parse, Compress, and Convert MIDI files into Numpy format
def process_metadata(midi_file):
    # Get outfile and check if it exists
    out_file = numpy_path/midi_file.relative_to(midi_path).with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): return
    
    npenc = transform_midi(midi_file)
    if npenc is not None: np.save(out_file, npenc)

#===============================================================

def transform_midi(midi_file):
    input_path = midi_file
    
    # Part 1: Filter out midi tracks (drums, repetitive instruments, etc.)
    try: 
#         if duet_only and num_piano_tracks(input_path) not in [1, 2]: return None
        input_file = compress_midi_file(input_path, min_variation=min_variation, cutoff=cutoff) # remove non note tracks and standardize instruments
        
        if input_file is None: return None
    except Exception as e:
        if 'badly form' in str(e): return None # ignore badly formatted midi errors
        if 'out of range' in str(e): return None # ignore badly formatted midi errors
        print('Error parsing midi', input_path, e)
        return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        print('Could not encode to chordarr:', input_path, e)
        print(traceback.format_exc())
        return None
    
    # Part 3. Compress song rests - Don't want songs with really long pauses 
    # (this happens because we filter out midi tracks).
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
    if delta_trim > 500: 
         print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
         return None
    chordarr = chord_short
    
    # Part 3. Chord array to numpy
    npenc = chordarr2npenc(chordarr)
    if not is_valid_npenc(npenc, input_path=input_path):
        return None
    
    return npenc

#===============================================================
def timeout_func(data, seconds):
    print("Timeout:", seconds, data.get('midi'))

#===============================================================
processed = process_all(process_metadata, midi_files, timeout=120, timeout_func=timeout_func)

In [ ]:
#@title Calculate and display number of resulting Numpy files
def create_databunch(files, data_save_name, path=data_path):
    save_file = path/data_save_name
    if save_file.exists():
        data = load_data(path, data_save_name)
    else:
        save_file.parent.mkdir(exist_ok=True, parents=True)
        vocab = MusicVocab.create()
        processors = [OpenNPFileProcessor(), MusicItemProcessor()]

        data = MusicDataBunch.from_files(files, path, processors=processors, encode_position=True)
        data.save(data_save_name)
    return data
numpy_files = get_files(numpy_path, extensions='.npy', recurse=False); len(numpy_files)

In [ ]:
#@title Create final training dataset file (training-data-piano-no-dynamics.pkl)
all_data = create_databunch(numpy_files, data_save_name=data_save_name); all_data

# TRAIN, SAVE, AND PLOT MODEL PERFOMANCE

In [ ]:
#@title Initialize the model
!nvidia-smi
batch_size = 32
encode_position = True
config = default_config()
dl_tfms = [batch_position_tfm] if encode_position else []
data = load_data(data_path, data_save_name, bs=batch_size, encode_position=encode_position, transpose_range=[0, 12], dl_tfms=dl_tfms)
learn = music_model_learner(data, config=config.copy(), metrics=[accuracy, error_rate])
!nvidia-smi


In [ ]:
#@title (Optional) Load Existing Model Checkpoint
config = default_config()
pretrained_path = Path('/content/musicautobot/SuperPianoModel.pth')
learn = music_model_learner(data, config=config, pretrained_path=pretrained_path, metrics=[accuracy, error_rate])

In [ ]:
#@title Find the optimum learning rate first by running this cell
learn.lr_find()
learn.recorder.plot()
learn.recorder.plot_lr(show_moms=True)

In [ ]:
#@title Start/Continue Training
number_of_training_epochs = 2 #@param {type:"slider", min:0, max:10, step:1}
optimum_learning_rate = 0.01 #@param {type:"number"}

learn.fit_one_cycle(number_of_training_epochs, optimum_learning_rate)

In [ ]:
#@title Save and Plot
learn.save('/content/musicautobot/SuperPianoModel')
learn.recorder.plot_lr(show_moms=True)

# GENERATE MUSIC WITH THE MODEL

## SETUP VARIABLES AND LOAD/RE-LOAD THE MODEL (just in case)

In [ ]:
#@title Setup Paths and Intialize the Model
# Location of your midi files
midi_path = Path('/content/musicautobot')

# Location of saved datset
data_path = Path('/content/musicautobot')

# Data and Vocab variables initialization
data = MusicDataBunch.empty(data_path) #for pretrained models
vocab = data.vocab

# Location of pretrained model
pretrained_path = Path('/content/musicautobot/SuperPianoModel.pth')
pretrained_path.parent.mkdir(parents=True, exist_ok=True)

# Initialize the model
config = default_config()
learn = music_model_learner(data, config=config, pretrained_path=pretrained_path)

### PREDICTION

## Choose existing midi file as a starting point

In [ ]:
#@title Upload your seed MIDI (will purge all MID/MIDI files you might've uploaded before)
%cd /content/musicautobot
!rm *.mid *.midi
uploaded = files.upload()
midi_files = get_files(midi_path, recurse=False, extensions='.mid'); midi_files[:5]
idx = 0
f = midi_files[idx]; f

## Trim (use seed from a MIDI file to predict next sequence/continuation)



In [ ]:
##@title Setup Pianoroll Output :) { run: "auto" }
graphs_length_inches = 18 #@param {type:"slider", min:0, max:20, step:1}
notes_graph_height = 5 #@param {type:"slider", min:0, max:20, step:1}

import librosa
import numpy as np
import pretty_midi
import pypianoroll
from pypianoroll import Multitrack, Track
import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('SVG')
# For plotting
import mir_eval.display
import librosa.display
%matplotlib inline


#midi_data = pretty_midi.PrettyMIDI('/content/MusicTransformer-Pytorch/output/rand.mid')

def plot_piano_roll(pm, start_pitch, end_pitch, fs=100):
    # Use librosa's specshow function for displaying the piano roll
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))



roll = np.zeros([int(graphs_length_inches), 128])


In [ ]:
#@title Show/Plot/Listen to the seed uploaded above. You can also run this cell to see Seed Continuation output below
MIDI_cutoff_beat = 20 #@param {type:"slider", min:0, max:100, step:1}
cutoff_beat = 20
item = MusicItem.from_file(f, data.vocab)
seed_item = item.trim_to_beat(cutoff_beat)

seed_item.stream.write('midi', '/content/musicautobot/seed_item.mid')

roll = np.zeros([int(graphs_length_inches), 128])
midi_data = pretty_midi.PrettyMIDI('/content/musicautobot/seed_item.mid')

plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, 24, 84)
plt.show(block=False)

play_wav(seed_item.stream)

## GENERATE (CHOOSE ONE OPTION BELOW)

### GENERATE w/starting sequence (Continuation)

In [ ]:
#@title Seed Continuation
number_of_tokens_to_generate = 256 #@param {type:"slider", min:0, max:4096, step:64}
max_model_temperature = 0.7 #@param {type:"slider", min:0, max:4, step:0.1}
min_model_temperature = 1.3 #@param {type:"slider", min:0, max:4, step:0.1}
model_top_k = 30 #@param {type:"slider", min:0, max:50, step:1}
model_top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}
minimum_number_of_bars_to_generate = 8 #@param {type:"slider", min:0, max:64, step:1}
seed_item = MusicItem.from_file('seed_item.mid', data.vocab)
pred, full = learn.predict(seed_item, n_words=number_of_tokens_to_generate, temperatures=(max_model_temperature, min_model_temperature), min_bars=minimum_number_of_bars_to_generate, top_k=model_top_k, top_p=model_top_p)
#pred.show()
pred.stream.write('midi', '/content/musicautobot/continuation.mid')


roll = np.zeros([int(graphs_length_inches), 128])
midi_data = pretty_midi.PrettyMIDI('/content/musicautobot/continuation.mid')

plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, 24, 84)
plt.show(block=False)

play_wav(pred.stream)

In [ ]:
#@title Would you like to add generated continuation to the seed/previous section??? If yes, run this cell, if not, go back and start over.
seed_item = seed_item.append(pred)
#seed_item.show()

seed_item.stream.write('midi', '/content/musicautobot/seed_continuation.mid')

roll = np.zeros([int(graphs_length_inches), 128])
midi_data = pretty_midi.PrettyMIDI('/content/musicautobot/seed_continuation.mid')

plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, 24, 84)
plt.show(block=False)

play_wav(seed_item.stream)

### GENERATE w/o a starting sequence (Performance)

In [ ]:
#@title Original Uninfluenced Performance
number_of_tokens_to_generate = 1024 #@param {type:"slider", min:0, max:4096, step:64}
max_model_temperature = 1.4 #@param {type:"slider", min:0, max:4, step:0.1}
min_model_temperature = 0.7 #@param {type:"slider", min:0, max:4, step:0.1}
model_top_k = 20 #@param {type:"slider", min:0, max:50, step:1}
model_top_p = 0.8 #@param {type:"slider", min:0, max:1, step:0.1}
minimum_number_of_bars_to_generate = 8 #@param {type:"slider", min:0, max:64, step:1}

empty_item = MusicItem.empty(vocab)
pred, full = learn.predict(empty_item, n_words=number_of_tokens_to_generate, temperatures=(max_model_temperature, min_model_temperature), min_bars=minimum_number_of_bars_to_generate, top_k=model_top_k, top_p=model_top_p)
#pred.show()
pred.stream.write('midi', '/content/musicautobot/performance.mid')

roll = np.zeros([int(graphs_length_inches), 128])
midi_data = pretty_midi.PrettyMIDI('/content/musicautobot/performance.mid')

plt.figure(figsize=[graphs_length_inches, notes_graph_height])
ax2 = plot_piano_roll(midi_data, 24, 84)
plt.show(block=False)

play_wav(pred.stream)